**Building a database for crime reports**


The goal of this project is to create a database named crimes_db with a table – boston_crimes – with appropriate datatypes for storing the data from the boston.csv file. I will be creating the table inside a schema named crimes. I will also create the readonly and readwrite groups with the appropriate privileges. Finally, I will also need to create one user for each of these groups.


Creating the Crime Database


In [2]:
#Creating the crime database
#we will start by using dq databade to create crime_db
import psycopg2
conn = psycopg2.connect(dbname = "dq", user = "dq")
conn.autocommit = True
cur= conn.cursor()
cur.execute("CREATE DATABASE crime_db;")



In [3]:
#connect to crime_db
conn = psycopg2.connect(dbname = "crime_db", user = "dq")

In [4]:
#create a schema
cur.execute("CREATE SCHEMA crimes")

 Obtaining the Column Names and Sample

In [5]:
#importing boston.csv
import csv
with open('boston.csv') as file:
    reader = csv.reader(file)
    col_header = next(reader)
    first_row = next(reader)

 Creating an Auxiliary Function

In [6]:
#creating a function that computes a python set with all 
#distinct values that helps identify the proper datatypes
#for each column

def get_col_set(csv_filename, col_index):
    distinct_values = set()

#csv_filename: the name of the CSV file
#col_index: the index of a column of that CSV file
    with open(csv_filename, 'r') as file:
        next(file)
        reader = csv.reader(file)
        for row in reader:
            distinct_values.add(row[col_index])

    return distinct_values

csv_filename = 'boston.csv'

for col_index in range(len(col_header)):
    distinct_values = get_col_set('boston.csv', col_index)
    num_values = len(distinct_values)
    print(col_header[col_index], num_values, sep='/t' )


incident_number/t298329
offense_code/t219
description/t239
date/t1177
day_of_the_week/t7
lat/t18177
long/t18177


**Finding the maximum length**


compute the maximum length of each value in the description column.

In [7]:
descriptions = get_col_set('boston.csv', 2)
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

58


**Creating the Table**


An enumerated datatype named weekday will be created for the day_of_the_week since there there are only seven possible values.

For the incident_number, we have decided to user the type INTEGER and set it as the primary key. The same datatype was also used to represent the offense_code.

Since the description has at most 58 characters, we decided to use the datatype VARCHAR(100) for representing it. This leaves some margin while not being so big that we will waste a lot of memory.

The date was represented as the DATE datatype. Finally, for the latitude and longitude, we used DECIMAL datatypes.

In [9]:
print(col_header)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [10]:
#create enum datatype
cur.execute("""
CREATE TYPE weekday as ENUM('Monday', 'Tuesday', 'Wednesday',
'Thursday', 'Friday', 'Saturday', 'Sunday')
            """)
conn.commit()

In [21]:
#creating table boston_crimes
cur.execute("""CREATE TABLE crimes.boston_crimes(
           incident_number INTEGER PRIMARY KEY,
           offense_code INTEGER,
           description VARCHAR(100),
           date DATE,
           day_of_the_week weekday,
           lat DECIMAL,
           long DECIMAL)""")
conn.commit()

Loading the Data

In [24]:
#loading the file
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER", f)
conn.commit()

Revoking Public Privileges

In [26]:
#revoking all privileges
cur.execute("REVOKE ALL ON SCHEMA public FROM public")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public")
conn.commit()

In [30]:
#creating users group readonly
cur.execute("CREATE GROUP readonly NOLOGIN")
cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readonly")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes to readonly")
conn.commit()


ProgrammingError: role "readonly" already exists


Creating User Groups

In [32]:
#creating users group readwrite
cur.execute("CREATE GROUP readwrite NOLOGIN")
cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readwrite")
cur.execute("GRANT SELECT INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes to readwrite")
conn.commit()

ProgrammingError: role "readwrite" already exists


Creating Users

In [38]:
#creating one user for each group
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1'")
cur.execute("GRANT readonly TO data_analyst")


conn.commit()

ProgrammingError: role "data_analyst" already exists


In [36]:
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2'")
cur.execute("GRANT readwrite TO data_scientist")

conn.commit()

In [40]:
# Close the old connection to test with a brand new connection.
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
# Check users and groups.
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
print()
# check privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.close()    

('data_analyst', False, False, False, True)
('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_scientist', False, False, False, True)

